In [95]:
import numpy as np
import scipy.linalg as la

def algorithm1(A):
    m, n = np.shape(A)
    Q = np.copy(A)
    R = np.zeros((n, n))
    for i in range(n):
        R[i, i] = la.norm(Q[:, i])
        Q[:, i] /= R[i, i]
        for j in range(i + 1, n):
            R[i, j] = Q[:, j]@Q[:, i]
            Q[:, j] -= R[i, j] * Q[:, i]
    return Q, R

A = np.random.random((6,4))
Q,R = la.qr(A, mode="economic")
print(A.shape, Q.shape, R.shape)
print(np.allclose(np.triu(R), R))
print(np.allclose(np.dot(Q.T, Q), np.identity(4)))
print(np.allclose(np.dot(Q, R), A))

Q,R = algorithm1(A)
print(A.shape, Q.shape, R.shape)
print(np.allclose(np.triu(R), R))
print(np.allclose(np.dot(Q.T, Q), np.identity(4)))
print(np.allclose(np.dot(Q, R), A))

(6, 4) (6, 4) (4, 4)
True
True
True
(6, 4) (6, 4) (4, 4)
True
True
True


In [80]:
def get_abs_det(A): return np.product(np.diagonal(la.qr(A, mode="economic")[1]))

In [81]:
def solver(A, b):
    m, n = np.shape(A)
    if m != n:
        raise ValueError('Please enter an n x n matrix.')
    Q, R = la.qr(A, mode="economic")
    y = Q.T@b
    x = np.zeros_like(y)
    for i, row in enumerate(reversed(R)):
        x[n - 1 - i] = (y[n - 1 - i] - row[n - i:]@x[n - i:]) / row[n - 1 - i]
    return x

In [88]:
def algorithm2(A):
    m, n = np.shape(A)
    R = np.copy(A)
    Q = np.identity(m)
    sign = lambda x: 1 if x >= 0 else -1
    for k in range(n):
        u = np.copy(R[k:,k])
        u[0] += sign(u[0]) * la.norm(u)
        u /= la.norm(u)
        R[k:,k:] -= np.outer(2 * u, u.T@R[k:,k:])
        Q[k:,:] -= np.outer(2 * u, u.T@Q[k:,:])
    return Q.T,R

A = np.random.random((5,3))
Q,R = algorithm2(A)
print(A.shape, Q.shape, R.shape)
np.allclose(Q.dot(R), A)

(5, 3) (5, 5) (5, 3)


True

In [93]:
def algorithm3(A):
    m, n = np.shape(A)
    H = np.copy(A)
    Q = np.identity(m)
    sign = lambda x: 1 if x >= 0 else -1
    for k in range(n - 2):
        u = np.copy(H[k+1:,k])
        u[0] += sign(u[0]) * la.norm(u)
        u /= la.norm(u)
        H[k+1:,k:] -= np.outer(2 * u, u.T@H[k+1:,k:])
        H[:,k+1:] -= np.outer(2 * (H[:,k+1:]@u), u.T)
        Q[k+1:,:] -= np.outer(2 * u, u.T@Q[k+1:,:])
    return H,Q.T

A = np.random.random((8,8))
H, Q = la.hessenberg(A, calc_q=True)
print(np.allclose(np.triu(H, -1), H))
print(np.allclose(np.dot(np.dot(Q, H), Q.T), A))

A = np.random.random((8,8))
H, Q = algorithm3(A)
print(np.allclose(np.triu(H, -1), H))
print(np.allclose(np.dot(np.dot(Q, H), Q.T), A))

True
True
True
True
